# Merge ZIP CSV Files into SQLite Database and Split by `measure_name`

This script processes multiple ZIP files, each containing a CSV file with the same name as the ZIP file.
It extracts the CSV files, merges their data into a single DataFrame, and writes the combined data into an SQLite database.
The data is split into multiple tables based on the unique values in the `measure_name` column.

## Dependencies
- Python 3.10.8
- Libraries: os, zipfile, pandas, sqlite3, multiprocessing

## Usage
1. Place this notebook in the same directory as the `../data/download` folder containing the ZIP files.
2. Run all cells in the notebook.
3. A SQLite database named `merged_data.db` will be created in the current directory.

## Output
- A SQLite database file (`merged_data.db`) containing multiple tables, one for each unique value in the `measure_name` column.
- Each table is named after the corresponding `measure_name` value.

In [1]:
import os
import zipfile
import pandas as pd
import sqlite3
from multiprocessing import Pool, cpu_count

# Define relative paths
zip_folder = os.path.join("../data", "download")  # Folder containing ZIP files
output_folder = "../data/database"  # SQLite database file path

# Function to process a single ZIP file and return its DataFrame
def process_zip_file(zip_file_name):
    try:
        zip_file_path = os.path.join(zip_folder, zip_file_name)
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            csv_file_name = os.path.splitext(zip_file_name)[0] + ".csv"
            if csv_file_name in zip_ref.namelist():
                extracted_path = zip_ref.extract(csv_file_name, path="temp")
                df = pd.read_csv(extracted_path)
                os.remove(extracted_path)
                # Drop columns with `_id` in their names
                df = df.drop(columns=[col for col in df.columns if "_id" in col])
                return df
            else:
                print(f"Warning: No CSV found in {zip_file_name}")
                return None
    except Exception as e:
        print(f"Error processing {zip_file_name}: {e}")
        return None
    
# Function to write a single measure's data to a database file
def write_measure_to_db(measure_data):
    measure_name, metric_name, sex_name, measure_df, output_folder = measure_data

    # Remove content within parentheses from measure_name
    measure_name_cleaned = measure_name.split('(')[0].strip().lower().replace(' ', '_').replace('-', '_')

    # Clean metric_name and sex_name
    metric_name_cleaned = metric_name.lower().replace(' ', '_').replace('-', '_')
    sex_name_cleaned = sex_name.lower().replace(' ', '_').replace('-', '_')

    # Generate the database file name
    db_file_name = f"{measure_name_cleaned}_{metric_name_cleaned}_{sex_name_cleaned}.db"
    db_file_path = os.path.join(output_folder, db_file_name)

    # Write the data to a new SQLite database file
    conn = sqlite3.connect(db_file_path)
    measure_df.to_sql("data", conn, if_exists="replace", index=False)
    conn.close()

    print(f"Created database: {db_file_name}")
    return db_file_name

In [2]:
# create temp folder to store extracted files
if not os.path.exists("temp"):
    os.makedirs("temp")

# Process all ZIP files in parallel
zip_files = [f for f in os.listdir(zip_folder) if f.endswith(".zip")]
pool_size = min(cpu_count(), len(zip_files))
print(f"Using {pool_size} processes...")

with Pool(pool_size) as pool:
    results = pool.map(process_zip_file, zip_files)

combined_df = pd.concat([df for df in results if df is not None], ignore_index=True)

if os.path.exists("temp"):
    os.rmdir("temp")

Using 42 processes...


In [3]:
# Split the combined DataFrame by `measure_name`, `metric_name`, and `sex_name`
unique_measures = combined_df["measure_name"].unique()
measure_data_list = []

for measure in unique_measures:
    measure_df = combined_df[combined_df["measure_name"] == measure]
    unique_metrics = measure_df["metric_name"].unique()
    for metric in unique_metrics:
        metric_df = measure_df[measure_df["metric_name"] == metric]
        unique_sexes = metric_df["sex_name"].unique()
        for sex in unique_sexes:
            sex_df = metric_df[metric_df["sex_name"] == sex]
            measure_data_list.append((measure, metric, sex, sex_df, output_folder))

# Set the pool size (default is the number of CPU cores)
pool_size = min(cpu_count(), len(measure_data_list))
print(f"Using {pool_size} processes to write database files...")

# Create a process pool and write database files in parallel
with Pool(pool_size) as pool:
    pool.map(write_measure_to_db, measure_data_list)

print("All data written to individual database files.")

Using 54 processes to write database files...
Created database: ylds_rate_both.db
Created database: ylds_rate_male.db
Created database: ylds_rate_female.db
Created database: ylds_number_male.db
Created database: ylds_number_female.db
Created database: ylds_number_both.db
Created database: ylds_percent_male.db
Created database: ylds_percent_female.db
Created database: ylds_percent_both.db
Created database: dalys_rate_male.db
Created database: dalys_rate_female.db
Created database: dalys_rate_both.db
Created database: dalys_number_male.db
Created database: dalys_number_female.db
Created database: dalys_number_both.db
Created database: dalys_percent_male.db
Created database: dalys_percent_female.db
Created database: dalys_percent_both.db
Created database: incidence_number_female.db
Created database: incidence_number_both.db
Created database: incidence_number_male.db
Created database: incidence_percent_male.db
Created database: incidence_percent_female.db
Created database: incidence_percen